In [ ]:
import numpy as np
from functools import wraps
from time import time

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func:%r args:[%r, %r] took: %2.4f sec' % \
          (f.__name__, args, kw, te-ts))
        return result
    return wrap


In [ ]:
#Initialise parameteres
S0 = 100  # initial stock price
K = 100   # strike price
T = 1    # time to maturity ( in years)
r = 0.08   # Risk-free rate (Annual)
N = 3      # number of time steps
u = 1.15   # up-factor in binomial models
d = 1/u    # down-factor
opttype = 'C' # Option Type 'C' for Call Option or 'P' for Put option

In [ ]:
import numpy as np

#Initialise parameteres
S0 = 100
K = 100   # strike price
T = 1      # time to maturity ( in years)
r = 0.08   # Risk-free rate (Annual)
N = 3      # number of time steps
u = 1.15   # up-factor in binomial models
d = 1/u    # down-factor
opttype = 'C' # Option Type 'C' for Call Option or 'P' for Put option


def american_binomial_tree_slow(K, T, S0, r, N, u, d, opttype="P"):
  # precompute constants for ease of calculation
  dt = T/N
  q = (np.exp(r*dt)-d)/(u-d)  # risk-neutral probability
  disc = np.exp(-r*dt)

  # initialising stock pricing at maturity - Time step N
  S = np.zeros(N+1)
  for j in range(0,N+1):
    S[j] = S0 * u**j * d**(N-j)

    #option payoff
    C = np.zeros(N+1)
    for j in range(0,N+1):
      if opttype == "P":
        C[j] = max(0, K - S[j])
      else:
        C[j] = max(0, S[j] - K)


        #backward recursion through the tree
        for i in np.arange(N-1,-1,-1):
         for j  in range(0,i+1):
          S = S0 * u**j * d**(i-j)
          C[j] = disc * (q * C[j+1] + (1-q) * C[j])
          if opttype == "P":
            C[j] = max(C[j], K - S)
          else:
            C[j] = max(C[j], S - K)
    return C[0]


In [ ]:
american_binomial_tree_slow(S0, K, T, r, N, u, d, opttype="P")

np.float64(99.34248376756801)

In [ ]:
import numpy as np


def american_binomial_tree_fast(K, T, S0, r, N, u, d, opttype="P"):
  # precompute constants for ease of calculation
  dt = T/N
  Q = (np.exp(r*dt)-d)/(u-d)  # risk-neutral probability
  DISC = np.exp(-r*dt)

  # initialising stock pricing at maturity - Time step N
  S = S0 * d**(np.arange(N,-1,-1)) * u**(np.arange(0,N+1,1))

  #option payoff
  if opttype == "P":
    C = np.maximum(0, K - S)
  else:
    C = np.maximum(0, S - K)


  #backward recursion through the tree
  for i in np.arange(N-1,-1,-1):
    S = S0 * d**(np.arange(i,-1,-1)) * u**(np.arange(0,i+1,1))
    C[:i+1]= DISC * (Q * C[1:i+2] + (1-Q) * C[0:i+1])
    C = C[:-1]
    if opttype == "P":
      C = np.maximum(C, K - S)
    else:
      C = np.maximum(C, S - K)
  return C[0]